In [16]:
import pandas as pd

#Correlation Matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
#Amado's csv to df 
#Don't run, you can just commit your own block 

#df_comments = pd.read_csv('/Users/amadouyehara/Desktop/Spring 2024/Applied Analytics & Predictive Modeling/Project 1/opiates_comments6_LIWC.csv')
#df_submissions = pd.read_csv('/Users/amadouyehara/Desktop/Spring 2024/Applied Analytics & Predictive Modeling/Project 1/opiates_submissions6_LIWC.csv', low_memory=False)


In [18]:
#Kaily's 

df_comments = pd.read_csv("C:/Users/Kaily Mejia/Dropbox/Graduate Courses/Applied and Predictive Modeling/Final Project/opiates_comments6_LIWC.csv" )
df_submissions = pd.read_csv('C:/Users/Kaily Mejia/Dropbox/Graduate Courses/Applied and Predictive Modeling/Final Project/opiates_submissions6_LIWC.csv', low_memory = False)


In [19]:
#Structure of Comments dataset
df_comments.head()

,author,created_utc,score,subreddit,WC,Analytic,Clout,Authentic,Tone,WPS,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,BoofDontShoot,1659067065,1,opiates,5,8.69,50.00,1.00,25.77,2.50,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
1,Busy_Background_195,1659067044,1,opiates,55,32.58,13.76,14.10,1.00,13.75,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,5.45,3.64,0.0
2,Wolf_Double,1659066949,1,opiates,6,1.00,95.20,23.51,99.00,6.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
3,Knowagoraaa,1659066905,1,opiates,39,71.92,60.13,99.00,73.64,13.00,...,0.0,0.0,0.0,0.0,0.0,2.56,0.00,5.13,0.00,0.0
4,Busy_Background_195,1659066875,1,opiates,75,24.37,29.68,14.56,2.41,25.00,...,0.0,0.0,0.0,0.0,0.0,0.00,2.67,4.00,0.00,0.0


In [20]:
#Structure of Submissions dataset
df_submissions.head()

,author,title,created_utc,score,subreddit,WC,Analytic,Clout,Authentic,Tone,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,Odd-Pizza-1079,How much tylenol is too much? Asking for a fri...,1.659067e+09,1,opiates,12.0,18.82,79.76,1.79,25.77,...,0.0,0.0,0.00,8.33,0.00,0.00,0.0,0.0,16.67,0.00
1,Global_Face_4708,Gabapentin/Lyrica/Clonidine Through Telehealth?,1.659065e+09,1,opiates,5.0,99.00,50.00,1.00,25.77,...,0.0,0.0,0.00,20.00,0.00,0.00,0.0,0.0,0.00,40.00
2,Bluebeamtwinfn,Opiates prolonging finish/enhancing sex,1.659064e+09,1,opiates,5.0,93.26,50.00,43.37,25.77,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,20.00
3,iLoveLuckiii,Boost your oxy high! &amp; Save money! (USE ON...,1.659063e+09,1,opiates,13.0,82.82,93.79,94.81,99.00,...,0.0,0.0,7.69,0.00,15.38,0.00,0.0,0.0,15.38,7.69
4,Downtown_Syrup4635,IOP Detox while working 9-5??,1.659061e+09,1,opiates,6.0,62.04,50.00,23.51,25.77,...,0.0,0.0,0.00,33.33,0.00,16.67,0.0,0.0,0.00,0.00


In [21]:
#Removing null values
print(len(df_comments))
df_comments = df_comments.dropna()
print(len(df_comments))

print(len(df_submissions))
df_submissions = df_submissions.dropna()
print(len(df_submissions))

1014397
1014397
79676
79674


In [22]:
#only using numerical values for heatmap 
'''
df_comments = df_comments.select_dtypes(include=['float64', 'int64'])

correlation_matrix = df_comments.corr()

plt.figure(figsize=(100, 80))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()
'''

'\ndf_comments = df_comments.select_dtypes(include=[\'float64\', \'int64\'])\n\ncorrelation_matrix = df_comments.corr()\n\nplt.figure(figsize=(100, 80))\nsns.heatmap(correlation_matrix, annot=True, cmap=\'coolwarm\', fmt=".2f")\nplt.title(\'Correlation Matrix Heatmap\')\nplt.show()\n'

In [23]:
#only using numerical values for heatmap 
'''
numerical_df_submissions = df_submissions.select_dtypes(include=['float64', 'int64'])


correlation_matrix = numerical_df_submissions.corr()

plt.figure(figsize=(100, 80))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()
'''

'\nnumerical_df_submissions = df_submissions.select_dtypes(include=[\'float64\', \'int64\'])\n\n\ncorrelation_matrix = numerical_df_submissions.corr()\n\nplt.figure(figsize=(100, 80))\nsns.heatmap(correlation_matrix, annot=True, cmap=\'coolwarm\', fmt=".2f")\nplt.title(\'Correlation Matrix Heatmap\')\nplt.show()\n'

In [24]:
df_comments = df_comments[['author', 'created_utc', 'score', 'subreddit', 'WC', 'Analytic', 'Clout', 'Authentic', 'Tone', 'WPS', 'Sixltr', 'Dic', 'pronoun', 'ppron', 'ipron', 'adverb', 'negate', 'verb', 'adj', 'compare', 'number', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'percept', 'feel', 'bio', 'health', 'sexual', 'ingest', 'risk', 'work', 'leisure', 'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent', 'nonflu', 'filler']]
df_submissions = df_submissions[['author', 'score', 'created_utc', 'pronoun', 'article', 'auxverb', 'adverb', 'conj', 'verb', 'adj', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'cogproc', 'focuspast', 'focuspresent', 'focusfuture', 'work','informal']]

In [25]:
df_submissions.head()

,author,score,created_utc,pronoun,article,auxverb,adverb,conj,verb,adj,...,negemo,anx,anger,sad,cogproc,focuspast,focuspresent,focusfuture,work,informal
0,Odd-Pizza-1079,1,1.659067e+09,0.00,8.33,16.67,25.00,8.33,25.00,0.00,...,0.0,0.0,0.0,0.0,16.67,0.0,16.67,0.0,0.00,0.0
1,Global_Face_4708,1,1.659065e+09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
2,Bluebeamtwinfn,1,1.659064e+09,0.00,0.00,0.00,0.00,0.00,20.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
3,iLoveLuckiii,1,1.659063e+09,7.69,7.69,0.00,7.69,7.69,15.38,7.69,...,0.0,0.0,0.0,0.0,7.69,0.0,15.38,0.0,0.00,0.0
4,Downtown_Syrup4635,1,1.659061e+09,0.00,0.00,0.00,0.00,16.67,16.67,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,16.67,0.0
